In [ ]:
import keras
from keras import backend, layers, models
from keras.datasets import mnist
import matplotlib.pyplot as plt

# 연쇄 방식 Modeling - OOP

In [ ]:
class CNN_seq_OOP(models.Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()
        self.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
        self.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
        self.add(layers.MaxPooling2D(pool_size=(2, 2)))
        self.add(layers.Dropout(0.25))
        self.add(layers.Flatten())
        self.add(layers.Dense(128, activation='relu'))
        self.add(layers.Dropout(0.5))
        self.add(layers.Dense(num_classes, activation='softmax'))
        self.compile(loss=keras.losses.categorical_crossentropy, optimizer='rmsprop', metrics=['accuracy'])

# Data - MNIST

In [ ]:
class Data_MNIST():
    def __init__(self):
        num_classes = 10

        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        img_rows, img_cols = x_train.shape[1:]

        if backend.image_data_format() == 'channels_first':
            x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
            x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
            input_shape = (1, img_rows, img_cols)
        else:
            x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
            x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
            input_shape = (img_rows, img_cols, 1)

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255

        y_train = keras.utils.to_categorical(y_train, num_classes)
        y_test = keras.utils.to_categorical(y_test, num_classes)

        self.input_shape = input_shape
        self.num_classes = num_classes
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test

# Draw Result

In [ ]:
def plot_loss(history, title=None):
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    if title is not None:
        plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Training', 'Validation'], loc=0)

def plot_acc(history, title=None):
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    if title is not None:
        plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Training', 'Validation'], loc=0)

# Usage

In [ ]:
EPOCHS = 10
BATCH_SIZE = 128

data = Data_MNIST()
model = CNN_seq_OOP(data.input_shape, data.num_classes)

history = model.fit(data.x_train, data.y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)

score = model.evaluate(data.x_test, data.y_test)
print(f"Test Loss {score[0]} and Accuracy {score[1]}")

plot_loss(history)
plt.show()
plot_acc(history)
plt.show()